In [1]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
import cmath
#import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%run "NN Audio Core.py"

In [3]:
# Round trip test with no NN evaluation to test pipeline
# Have to get phase information from the noisy file to match what happens for real

file = "p232_013.wav"
clean_file = "Assets\\DataShareArchive\\Test\\Clean\\" + file
noisy_file = "Assets\\DataShareArchive\\Test\\Noisy\\" + file
wav, rate = librosa.core.load(clean_file)
noisy_wav, rate = librosa.core.load(noisy_file)

In [74]:
ft = get_ft(wav)
print(ft.shape)
swav = inv_ft(ft)
Audio(swav,rate=22050)

(513, 340)


In [10]:
Audio(wav, rate=22050)

In [72]:
print(FMIN, N_FFT)
freq = librosa.fft_frequencies(sr=22050, n_fft=N_FFT)
print(freq.shape)
print(freq[300], freq[3])

32.70319566257483 1024
(513,)
6459.9609375 64.599609375


In [124]:
import math

# Low cut off ~65hz - seems like 125hz should be safe but keeping the low end
# High cut off ~6500hz - 5000hz sounded OK, but keeping some just in case
# These are based on N_FFT 1024
LOW_BIN = 3
HIGH_BIN = 300
SAMPLE_BINS = HIGH_BIN-LOW_BIN

# Number of bins in rescaled pitch representation of the ft
# No idea - need to test
PITCH_BINS = 200

# S = sample space, which frequency  P = pitch space, which is rescaled so pitches are constant space appart
def S_ix(p_ix):
    return (SAMPLE_BINS)*(2**(p_ix/PITCH_BINS) - 1)

def P_ix(s_ix):
    return PITCH_BINS * math.log(1+s_ix/(SAMPLE_BINS),2)

# Super dumb sampler that just takes the closest fit
def samples_to_pitch(bins, p_ix):
    s_ix = math.floor(S_ix(p_ix))
    return bins[s_ix]

def pitch_to_samples(pt, s_ix):
    p_ix = math.floor(P_ix(s_ix))
    return pt[p_ix]

# Takes samples and rescales to pitch (log of frequencies so space between same pitch is constant)
# Output is samples, then rescaled magnitudes in shape (sample_count, PITCH_BINS)
def pitch_scale(ft):
    pt = np.empty((ft.shape[0], PITCH_BINS))
    for s in range(ft.shape[0]):
        for i in range(PITCH_BINS):
            pt[s,i] = samples_to_pitch(ft[s,:],i)
    return pt

def sample_scale(pt):
    ft = np.empty((pt.shape[0], SAMPLE_BINS))
    for s in range(pt.shape[0]):
        for i in range(SAMPLE_BINS):
            ft[s,i] = pitch_to_samples(pt[s,:],i)
    return ft

# Sample output is ft converted to magnitude and .T to get samples then ft
# Only returns 'interesting' samples - between LOW_ and HIGH_ bins, so output shape is (sample_count, SAMPLE_BINS)
def get_samples(file):
    wav, rate = librosa.core.load(file)
    samples = abs(get_ft(wav).T) # organized as bins, frames so we need to transpose them to frames, bins
    return samples[:,LOW_BIN:HIGH_BIN]

def rebuild_fft(output, original_fft):
    fft = np.zeros((output.shape[0], FFT_BINS))
    fft[:,LOW_BIN:HIGH_BIN] = output
    vphase = np.vectorize(cmath.phase)
    o_phase = vphase(original_fft)
    mag = fft.T
    vrect = np.vectorize(cmath.rect)
    return vrect(mag, o_phase)

In [125]:
print(ft.shape)
samples = get_samples(clean_file)
print(samples.shape)
pt = pitch_scale(samples)
print(pt.shape)
s2 = sample_scale(pt)
print(s2.shape)
wav = rebuild_fft(s2, noisy_ft)

(513, 340)
(340, 297)
(340, 200)
(340, 297)


In [126]:
samples = get_samples(clean_file)

print(samples.shape)

noisy_ft = get_ft(noisy_wav)

pt = pitch_scale(samples)
s2 = sample_scale(pt)
rt = rebuild_fft(s2, noisy_ft)
rwav = inv_ft(rt)
Audio(rwav,rate=22050)

(340, 297)


In [32]:
Audio(wav,rate=22050)

In [47]:
rect_vect = np.vectorize(cmath.rect)
rt = rect_vect(M,P)

print (rt[10,10], ft[10,10])

(0.020266860723495483+0.026656782254576687j) (0.02026686+0.026656782j)


In [33]:
noisy_ft = get_ft(noisy_wav)
noisy_ft.shape

r = noisy_ft.real
i = noisy_ft.imag
samples = np.empty((r.shape[0], r.shape[1],2))
samples[:,:,0] = r
samples[:,:,1] = i

print(samples.shape)


(768, 340, 2)


In [41]:


s = get_samples(noisy_file)

f = rebuild_cqt(s)

print(f[20,10], noisy_ft[20,10])
print(f.shape, noisy_ft.shape)

(0.04094690456986427-0.023657120764255524j) (0.040946905-0.02365712j)
(768, 340) (768, 340)


In [29]:
print ( samples.shape )

(340, 768, 2)


In [3]:
noisy_ft = get_ft(noisy_wav)
clean_samples = get_samples(clean_file)

rt_ft = rebuild_fft(clean_samples, noisy_ft)

rt_wav = inv_ft(rt_ft)
print(rt_ft.shape)

display_fft(rt_ft)
Audio(rt_wav,rate=22050)

NameError: name 'get_samples' is not defined

In [ ]:
# Compare to clean
display_fft(get_ft(wav))
Audio(wav,rate=22050)

In [ ]:
# Compare to noisy
display_fft(noisy_ft)
Audio(noisy_wav,rate=22050)

In [ ]:
wav, rate = librosa.core.load("Assets\\DataShareArchive\\Test\\clean\\p232_011.wav")

#draw("p232_010.wav", "clean")

c_f = get_ft(wav)

def filter(cqt):
    cqt[0:BINS_PER_OCTAVE,:] = 0
    return cqt

c_f = filter(c_f)

librosa.display.specshow(librosa.amplitude_to_db(np.abs(c_f), ref=np.max), y_axis='cqt_hz')

print(c_f.shape)

rewav = inv_ft(c_f)

Audio(rewav, rate=rate)


In [ ]:
Audio(wav, rate=rate)

In [ ]:
n_f = draw("p232_010.wav", "noisy")

In [ ]:
# CQT experiments

hop_length = 256
bins_per_octave = 12 * 8
fmin = librosa.note_to_hz('C1')
octaves = 8
C = librosa.cqt(wav, hop_length=hop_length, fmin=fmin, n_bins=octaves*bins_per_octave, bins_per_octave=bins_per_octave)
print(C.shape)

C[0:bins_per_octave,:] = 0
fade = bins_per_octave//2
for i in range(0,fade):
    C[bins_per_octave+i,:] = (i/fade) * C[bins_per_octave+i,:]

rewav = librosa.icqt(C, hop_length=hop_length, bins_per_octave=bins_per_octave)


librosa.display.specshow(librosa.amplitude_to_db(np.abs(C), ref=np.max), y_axis='cqt_note', x_axis='time', hop_length=hop_length, fmin=fmin,  bins_per_octave=bins_per_octave)


Audio(rewav, rate=rate)


In [ ]:
# polar experiments
import cmath

n_f_m = abs(n_f)
c_f_m = abs(c_f)
vphase = np.vectorize(cmath.phase)
x = vphase(n_f)
print(x)

mag_ratio = c_f_m/n_f_m

re_f = n_f * mag_ratio

rewav = librosa.istft(re_f, hop_length=256)

Audio(rewav, rate=rate)

# this works for full complex numbers
#diff = n_f - c_f
#reclean = n_f - diff
#real_clean = c_f.real + 1j * c_f.imag


In [ ]:
Audio(wav, rate=rate)


In [ ]:
librosa.display.specshow(librosa.amplitude_to_db(np.abs(diff), ref=np.max), y_axis='log', x_axis='time')

In [ ]:
round_trip = librosa.istft(real_clean)
Audio(round_trip, rate=rate)

In [ ]:
round_trip = librosa.istft(X)
Audio(round_trip, rate=rate)

In [59]:
(3.14/2 + 3.14/2) % 6.28

3.14